In [ ]:
import numpy as np 
import math
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt

from numba import jit,njit, prange


In [ ]:
sigma   = 1
L =  100
npart = 32
pressure = 1
dL_max = 0.05
dx = 1.0/16
halfL = L/2

#z=1 #exp(mu)

In [ ]:
# Some helper functions
@jit
def wrap(particle):
    '''Apply perodic boundary conditions.'''
    if particle >= L:
        particle -= L
    elif particle < 0:
        particle += L
    return particle

In [ ]:
@jit
def Vext_cal(x,a,b,c):
    V=0
    return V    

In [ ]:
x= np.linspace(0,L,32//dx)
Vext = np.zeros(int(32/dx))
for i in range (int(32/dx)):
    Vext[i]=Vext_cal(i*L/32*dx,1,L/4,3)
plt.plot(x,Vext)


In [ ]:
@jit
def distancesq(x1, x2,L):
    '''Gets the squared distance between two particles, applying the minimum image convention.'''
    # Calculate distances
    Dx = np.abs(x1-x2)
    if Dx > L/2:
        Dx -= L
    return Dx**2

In [ ]:
@jit
def energy(particles,epsilon,a,b,c,L):
    '''Gets the energy of the system'''
    energy = 0.
    
    for i in range(npart):
        for j in range(npart):
            if(i!=j):
                dist = distancesq(particles[i], particles[j],L)
                if(dist**0.5<16):
                    energy += 4.*epsilon*((1/dist**6)-(1/dist**3))
                if(dist<1):
                    return 10**8
    energy/=2
    
    for i in range(npart):
        energy+=Vext_cal(particles[i],a,b,c)
    
    return energy 

In [ ]:
@jit
def particleEnergy(particle,p,particles,epsilon,a,b,c,L):
    '''Gets the energy of a single particle.'''
    part_energy = 0.
    
    for i in range(npart):
        if(i!=p):
            dist = distancesq(particle, particles[i],L)
            if(dist**0.5<16):
                part_energy += 4*epsilon*((1/dist**6)-(1/dist**3))
            if(dist<1):
                return 10**8
    
    part_energy+=Vext_cal(particle,a,b,c)
    
    return part_energy

In [ ]:
# Initialize the simulation box:
@jit
def initial(L):
    particles = np.zeros(npart)
    
    for i in range(npart):
        x_coord = np.random.uniform(0, L)
        particles[i]=i*L/npart
        #prop = random.uniform(0, 1)
        #if(prop>=0.5):
        #    exist[i]=1
    return particles

In [ ]:
@jit
def MC_step(particles,L,pressure,steps,eps,a,b,c,Print=True):
    for step in range(0, steps):
        prop = np.random.uniform(0, 1)#volumn or move
        if (prop*(npart+1) + 1 < npart):
            for i in range(npart):
                # Move particle and evaluate energy
                this_particle = np.copy(particles[i])
                prev_E = particleEnergy(this_particle,i, particles, eps,a,b,c,L)
                this_particle =np.random.uniform(0, L)
                new_E =  particleEnergy(this_particle,i, particles, eps,a,b,c,L)
                deltaE = new_E - prev_E
                rand = np.random.uniform(0,1)
                #if(deltaE<10**3):
                if (np.exp(-deltaE) > rand):
                    particles[i] = this_particle
                
        else:
            oldL =  L
            rand = np.random.uniform(-0.5, 0.5)
            #Suggest a random volume by preforming a random walk in lnV.
            lnLTrial = np.log(oldL*(1+rand*dL_max)) 
            LTrial = np.exp(lnLTrial)
            newL = LTrial
            coordsTrial = particles*(newL/oldL)
            energy_old = energy(particles,eps,a,b,c,oldL)
            energy_new = energy(coordsTrial,eps,a,b,c,newL)
            weight=(energy_new - energy_old) + pressure*(newL - oldL) - (npart+1)*np.log(newL/oldL)
            
            rand = np.random.uniform(0, 1)
            #if(weight<10**3):
            if (rand < np.exp(-weight)):
                L = newL
                particles*=(newL/oldL)
                #if(L<npart):
                    #print(particles,L)
    return particles,energy(particles,eps,a,b,c,L),L

In [ ]:
@jit
def MC_result(seed,L,pressure,eps,a,b,c):
    np.random.seed(seed)
    particles = initial(L)
    "warm up"
    particles, ene ,L= MC_step(particles,L,pressure,10**5,eps,a,b,c)
    print("WARM UP",ene,L)
    avg_step = 10**5
    ene_array = np.zeros(0)
    L_array = np.zeros(0)
    ene_array=np.append(ene_array,energy)
    L_array=np.append(L_array,L)
    for i in range(avg_step-1):
        np.random.seed(i)
        particles, ene,L = MC_step(particles,L,pressure,512,eps,a,b,c,Print=False)
        ene_array=np.append(ene_array,ene)
        L_array=np.append(L_array,L)
        #rho += density_cal(particles)

        #if(i%1000==0):
        #    print(i,ene,L)
    #rho=rho/avg_step
    #print("energy_avg",np.sum(ene_array)/len(ene_array),
    #      np.sum(L_array)/len(L_array))
    return particles,ene_array,L_array

@jit
def density_result(seed,L,pressure,eps):
    """
    density = random.uniform(0.3,0.5)
    a=random.uniform(1,3)
    b=random.uniform(0,1)
    c=random.uniform(1,L/8)
    """
    a=0
    b=0
    c=0
    particles,energy,L_array = MC_result(seed,L,pressure,eps,a,b,c)
    return particles,energy,L_array

In [ ]:

def run(L):
    N=30
    P_array=np.exp(np.linspace(np.log(0.05),np.log(10),N))
    print(P_array)
    rho_array=np.zeros(N)
    phase=1
    eps = phase*0.5
    print("eps\t=\t",eps)
    file = "P_rho_eps_"+str(eps)+".dat"
    with open(file, "w") as text_file:
        for i in range (N):
            print(i,"/",N)
            particles,energy_array,L_array=density_result(123,L,P_array[i],eps)
            rho0=np.mean(npart/L_array)
            rho1=npart/np.mean(L_array)
            L=np.mean(L_array)
            text_file.write(str(P_array[i])+'\t'+str(rho0)+'\t'+str(rho1)+"\n")
            print(str(P_array[i])+'\t'+str(rho0)+'\t'+str(rho1)+"\n")
            if(rho0>0.85):
                break;

In [ ]:
run(L)